In [1]:
import numpy as np
from scipy.stats import uniform, norm, f, t

Случайный вектор $(\xi_1, \xi_2, \xi_3, \xi_4, \xi_5, \eta)$ имеет компоненты, распределённые по следующему закону: $\xi_i \sim R(-1, 1)$, $\eta \sim N(2 + 3 x_1 - 2 x_2 + x_3 + x_4 - x_5, 1.5^2)$, где $x_i$ это значения, которые принимает случайная величина $\xi_i$. Сгенерировать выборку объёма $n = 50$.

In [2]:
np.random.seed(15)
n = 50
m = 5
X = np.hstack((np.ones((n, 1)), uniform(-1, 2).rvs(size=(n, m))))
b_true = np.array([[2, 3, -2, 1, 1, -1]])
y = norm(X @ b_true.T, 1.5).rvs()

**a**. Проверить переменные $\xi_i$ на мультиколлинеарность

In [3]:
for i in range(1, m):
    for j in range(i + 1, m + 1):
        print(f'r{i}{j} = %.4f' % np.corrcoef(X[:,i], X[:,j])[1, 0])

r12 = -0.1101
r13 = 0.0386
r14 = -0.0425
r15 = -0.0990
r23 = 0.1040
r24 = -0.0514
r25 = -0.0823
r34 = -0.0801
r35 = 0.0444
r45 = 0.0910


In [111]:
def get_R2(y, y_true):
    RSS0 = ((y_true - y_true.mean()) ** 2).sum()
    RSS = ((y_true - y) ** 2).sum()
    return (RSS0 - RSS)/RSS0

In [112]:
def regression_p_val(R2, n, p):
    delta = R2 / (1 - R2) * (n - p) / (p - 1)
    return 1 - f(p - 1, n - p).cdf(delta)

In [113]:
for i in range(1, m + 1):
    X_i = np.delete(X, i, axis=1)
    y_i = X[:, i].T
    
    F_i = X_i.T @ X_i
    b_i = np.linalg.inv(F_i) @ X_i.T @ y_i

    R2_i = get_R2(X_i @ b_i, y_i)
    
    m_i = m - 1
    p_i = m_i + 1 # (m_i + 1) т. к. учитываем коэффициент сдвига

    pval_i = regression_p_val(R2_i, n, p_i)
    print(f'x{i}: R2 = %.4f, pval = %.4f' % (R2_i, pval_i))

x1: R2 = 0.0282, pval = 0.8585
x2: R2 = 0.0352, pval = 0.8005
x3: R2 = 0.0228, pval = 0.9001
x4: R2 = 0.0178, pval = 0.9348
x5: R2 = 0.0295, pval = 0.8483


**b**. Определить уравнение линейной регрессии $\eta = \beta_0 + \sum_{i = 1}^5 \beta_i \xi_i$ и проверить значимость коэффициентов

In [114]:
F = X.T @ X
F_inv = np.linalg.inv(F)
b = F_inv @ X.T @ y
print(*('%.4f' % x for x in b.flatten()))

2.0092 2.9520 -2.3268 1.1635 1.3157 -0.4425


In [115]:
y_pred = X @ b

Значимость коэффициентов:

In [116]:
for i in range(m + 1):
    p_i = m + 1 # (m + 1) т. к. учитываем коэффициент сдвига
    RSS_i = ((y - y_pred) ** 2).sum()
    delta_i = b[i, 0] / (RSS_i * F_inv[i, i]) ** 0.5 * (n - p_i) ** 0.5
    
    pval_i = 2 * (1 - t(n - p_i).cdf(abs(delta_i)))
    print(f'b{i}: pval = %.4e' % pval_i)

b0: pval = 5.7732e-15
b1: pval = 5.5511e-15
b2: pval = 2.6753e-10
b3: pval = 2.7879e-04
b4: pval = 5.5776e-05
b5: pval = 1.6549e-01


**c**. Определить коэффициент детерминации и проверить его значимость

In [117]:
R2 = get_R2(y_pred, y)
print('R2 = %.4f' % R2)

R2 = 0.8552


In [118]:
p = m + 1 # (m + 1) т. к. учитываем коэффициент сдвига
pval = regression_p_val(R2, n, p)
print(f'pval = %.4e' % pval)

pval = 1.1102e-16


**d**. Найти значение в точке $x_i = 0$ и построить $95\%$ доверительный интервал

In [12]:
#значение в нуле равно значению свободного члена:
x = np.array([1, 0, 0, 0, 0, 0])
print(*x @ b)

2.0092024505973427


In [13]:
alpha = 0.95
RSS = ((X @ b - y) ** 2).sum()
delta = t(n - p).interval(alpha)[1] * ((1 + (x @ F_inv @ x.T)) * RSS) ** 0.5 / (n - p) ** 0.5

print(b[0, 0] - delta, b[0, 0] + delta)

-0.3254762009569019 4.343881102151587


**e**. Проверить предположение о независимости ошибок

In [14]:
eps = (X @ b - y).T[0]
I = 0
for i in range(eps.shape[0] - 1):
    I += (eps[:i] > eps[i]).sum()

In [15]:
I = sum((eps[i + 1:] < eps[i]).sum() for i in range(eps.shape[0]))

In [16]:
delta = (I - 0.25 * n * (n - 1)) / (n ** 3 / 36) ** 0.5
pval = 2 * (1 - norm(0, 1).cdf(abs(delta)))
print('pval = %.4f' % pval)

pval = 0.7925


**f**. Проверить предположение о нормальности распределения ошибок

In [17]:
eps.sort()

In [18]:
mu = eps.mean()
sigma = (((eps - mu) ** 2).sum() / (n - 1)) ** 0.5

In [19]:
delta = np.abs(norm(mu, sigma).cdf(eps) - (np.arange(n) + 1) / n).max()

In [127]:
N = 200000
sample = norm(0, 1).rvs(size=(N, n))
sample.sort(axis=1)
mu_i = sample.mean(axis=1)
sigma_i = (((sample - mu_i.reshape(-1, 1)) ** 2).sum(axis=1) / (n - 1)) ** 0.5
delta_i = np.abs(norm(mu_i, sigma_i).cdf(sample.T).T - (np.arange(n) + 1) / n).max(axis=1)
pval = (delta_i > delta).sum() / N
print('pval = %.4f' % pval)

pval = 1.0000


**g**. Провести кросс-валидацию регрессии.

In [21]:
RSS_i = np.zeros(n)
for i in range(n):
    y_i = np.delete(y, i, axis=0)
    X_i = np.delete(X, i, axis=0)
    F_i = X_i.T @ X_i
    b_i = np.linalg.inv(F_i) @ X_i.T @ y_i
    RSS_i[i] = (X[i] @ b_i - y[i]) ** 2

In [22]:
RSS = RSS_i.sum()
RSS0 = ((y - y.mean()) ** 2).sum()
print('R2_cv = %.4f' % ((RSS0 - RSS) / RSS0))
print('R2 = %.4f' % R2)

R2_cv = 0.8127
R2 = 0.8552


**h**. Проверить адекватность регрессии, сделав 5 повторных измерений в одной точке.

In [46]:
k = 5
x = np.array([[1, 0.5, 0, 0, 0, 0]])
y_rep = norm(x @ b_true.T, 1.5).rvs(k) 

In [47]:
sigma_rep_2 = ((y_rep - y_rep.mean()) ** 2).sum() / (k - 1)
p = m + 1 # (m + 1) т. к. учитываем коэффициент сдвига
pval = 1 - f((n - p), (k - 1)).cdf(((y_pred - y) ** 2).sum() / (sigma_rep_2 * (n - p)))
print('pval = ', pval)

pval =  0.6284765396746522


**i**. Удалить переменную, соответствующую наименее значимому коэффициенту и повторить пункты b и c, сравнить уравнения регрессии.

In [56]:
X_new = np.delete(X, 5, axis = 1)
m_new = m - 1

In [57]:
F_new = X_new.T @ X_new
F_new_inv = np.linalg.inv(F_new)
b_new = F_new_inv @ X_new.T @ y
print(*('%.4f' % x for x in b_new.flatten()))

1.9638 2.9909 -2.2879 1.1362 1.2795


In [58]:
y_new_pred = X_new @ b_new

Значимость коэффициентов:

In [61]:
for i in range(m_new + 1):
    p_i = m_new + 1 # (m + 1) т. к. учитываем коэффициент сдвига
    RSS_i = ((y - y_new_pred) ** 2).sum()
    delta_i = b[i, 0] / (RSS_i * F_new_inv[i, i]) ** 0.5 * (n - p_i) ** 0.5
    
    pval_i = 2 * (1 - t(n - p_i).cdf(abs(delta_i)))
    print(f'b{i}: pval = %.4e' % pval_i)

b0: pval = 3.3307e-15
b1: pval = 4.4409e-15
b2: pval = 2.6897e-10
b3: pval = 3.0274e-04
b4: pval = 5.9670e-05


Коэффициент детерминации и его значимость

In [107]:
R2_new = get_R2(y_new_pred, y)
print('R2_new = %.4f' % R2)

R2_new = 0.8487


In [108]:
p_new = m_new + 1 # (m + 1) т. к. учитываем коэффициент сдвига
pval = regression_p_val(R2_new, n, p_new)
print(f'pval = %.4e' % pval)

pval = 1.1102e-16


**j**. Сравнить уравнения регрессии бутстрепом

In [125]:
N = 200000
index = np.random.randint(0, n, size=(N, n))
delta = (R2 - R2_new) / R2
deltas = np.zeros(N)
for i, X_i, y_i in zip(range(N), X[index], y[index]):
    F_i = X_i.T @ X_i
    F_i_inv = np.linalg.inv(F_i)
    b_i = F_i_inv @ X_i.T @ y_i
    R2_i = get_R2(X_i @ b_i, y_i)
    
    X_i_new = np.delete(X_i, 5, axis=1)
    
    F_i_new = X_i_new.T @ X_i_new
    F_i_new_inv = np.linalg.inv(F_i_new)
    b_i_new = F_i_new_inv @ X_i_new.T @ y_i
    R2_i_new = get_R2(X_i_new @ b_i_new, y_i)
    
    deltas[i] = (R2_i - R2_i_new) / R2_i
print('pval = %.4f' % (deltas > delta).mean())

pval = 0.4398
